# Group Progject Part 3

Ananov Georgy - 890980   
Le Duong - 894834  
Nguyen Tam - 906968  
Pham Hieu - 895118  

## Library Imports

In [ ]:
import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine, text
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

## Database connection

First things first, we need to connect to the *PostgreSQL* database we designed and implemented during the previous rounds. We can use methods from the `psycopg2` and `sqlalchemy` libraries to establish a connection and execute SQL queries.

In [ ]:
# Credentials to connect to Postgres database
database="grp21_vaccinedist"   
user='grp21'       
password='B5!BpWYT' 
host='dbcourse2022.cs.aalto.fi'
port= '5432'

try:
    connection = psycopg2.connect(
                                    database=database,              
                                    user=user,       
                                    password=password,   
                                    host=host,
                                    port=port
                                )
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
    DIALECT = 'postgresql+psycopg2://'
    db_uri = "%s:%s@%s/%s" % (user, password, host, database)
    engine = create_engine(DIALECT + db_uri)
    psql_conn  = engine.connect()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

With the connection established, we can proceed to conducting the analysis tasks.

## Data analysis

### Question 1

Let's start by pulling data about patients and symptoms from the database into a pandas dataframe using an SQL query.

In [ ]:
query = '''
    SELECT ssno, gender, birthday, symptom, date
    FROM patient, diagnosed
    WHERE patient = ssno
'''
df = pd.read_sql(query, psql_conn)
df.head()


Now we can create a table using the dataframe we obtained.

In [ ]:
df.to_sql("patientsymptoms", psql_conn, index = True, if_exists = "replace")


We can check that the table has been successfully inserted into the database.

In [12]:
query = '''
    SELECT *
    FROM patientsymptoms
'''
df = pd.read_sql(query, psql_conn)
df.head()

,index,ssno,gender,birthday,symptom,date
0,0,790503-394M,F,1979-05-03,anaphylaxia,2021-01-30
1,1,000127-4899,F,2000-01-27,blurring of vision,2021-02-01
2,2,090707-295R,F,2009-07-07,chest pain,2021-02-15
3,3,810616-9029,M,1981-06-16,chest pain,2021-04-30
4,4,701127-5340,F,1970-11-27,chills,2021-04-10


### Question 2

For this question we can start by creating a view that lists some information about every performed vaccination.

In [29]:
%%capture

query = '''
    CREATE OR REPLACE VIEW individualvaccination AS
    SELECT attend.patient AS patientssno, attend.date, vaccinedata.name as vaccinetype
    FROM   
        attend 
        NATURAL JOIN 
        vaccinationevent 
        NATURAL JOIN 
        vaccinationbatch 
        NATURAL JOIN 
        vaccinedata
    
'''
psql_conn.execute(query)


We can now use the created view in constructing the desired dataframe.

In [30]:
query = '''
    SELECT ssno AS patientssno, IV1.date AS date1, IV1.vaccinetype ASvaccinetype1,
        IV2.date AS date2, IV2.vaccinetype ASvaccinetype2
    FROM 
        patient
        LEFT JOIN individualvaccination AS IV1 ON ssno = IV1.patientssno
        LEFT JOIN individualvaccination AS IV2 ON ssno = IV2.patientssno AND IV1.date <> IV2.date
'''

df = pd.read_sql(query, psql_conn)
df.head()

,patientssno,date1,date2,asvaccinetype1,asvaccinetype2
0,841229-112N,2021-01-30,2021-02-14,AstraZeneca,Comirnaty
1,780214-1893,2021-01-30,None,AstraZeneca,None
2,731122-126T,2021-01-30,None,AstraZeneca,None
3,160930-586P,2021-01-30,2021-02-14,AstraZeneca,Comirnaty
4,930106-189U,2021-01-30,None,AstraZeneca,None


### Question 3

### Question 4

### Question 5

### Question 6

### Question 7

### Question 8

### Question 9

### Question 10